In [2]:
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import KFold,train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [3]:
file=r'E:\lanchan VIP files\data scienece\python\Data\bank-full.csv'
bd=pd.read_csv(file,delimiter=';')

In [5]:
file=r"E:\lanchan VIP files\data scienece\python\Data\census_income.csv"

cd=pd.read_csv(file)

cd

cd['Y'].value_counts()

cd['Y']=(cd['Y']==' >50K').astype(int)

cd['Y'].value_counts()

pd.crosstab(cd['education'],cd['education.num'])

del cd['education']

cd.select_dtypes(['object']).nunique()

cd['occupation'].value_counts()

cd['native.country'].value_counts()

catcols=cd.select_dtypes(['object']).columns

for col in catcols:
    freqs=cd[col].value_counts()
    k=freqs.index[freqs>99][:-1]
    for cat in k:
        name=col+'_'+cat
        cd[name]=(cd[col]==cat).astype(int)
    del cd[col]
    print(col)

cd.shape

cd_train,cd_test=train_test_split(cd,test_size=0.2,random_state=2)

cd_train.reset_index(drop=True,inplace=True)
cd_test.reset_index(drop=True,inplace=True)


x_train=cd_train.drop(["Y"],axis=1)
y_train=cd_train['Y']

x_test=cd_test.drop(["Y"],axis=1)
y_test=cd_test['Y']

x_train.shape

workclass
marital.status
occupation
relationship
race
sex
native.country


(26048, 48)

In [6]:
cd_train

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,Y,workclass_ Private,workclass_ Self-emp-not-inc,workclass_ Local-gov,...,race_ Amer-Indian-Eskimo,sex_ Male,native.country_ United-States,native.country_ Mexico,native.country_ ?,native.country_ Philippines,native.country_ Germany,native.country_ Canada,native.country_ Puerto-Rico,native.country_ El-Salvador
0,31,247444,11,0,0,40,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,37,34996,9,0,0,40,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,18,194059,8,0,0,40,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
3,38,224566,11,0,0,50,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
4,30,363296,9,0,0,72,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0
5,18,126154,10,0,0,40,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
6,53,281425,9,0,0,40,1,1,0,0,...,0,1,1,0,0,0,0,0,0,0
7,41,208470,9,0,0,40,1,1,0,0,...,0,1,1,0,0,0,0,0,0,0
8,50,161075,9,0,0,40,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,33,104509,9,0,0,40,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [15]:
clf1=KNeighborsClassifier(n_neighbors=50)
clf2=RandomForestClassifier(class_weight='balanced',n_estimators=200)
clf3=RandomForestClassifier(class_weight=None,n_estimators=100)
clf4=XGBClassifier(n_estimators=150,objective='binary:logistic',learning_rate=.01)
clf5=XGBClassifier(n_estimators=150,objective='binary:logistic',learning_rate=.1)

algos=[clf1,clf2,clf3,clf4,clf5]

In [16]:
rows=x_train.shape[0]

In [17]:
layer1=pd.DataFrame({'clf1':np.zeros(rows),'clf2':np.zeros(rows),'clf3':np.zeros(rows),
                    'clf4':np.zeros(rows),'clf5':np.zeros(rows)})

In [18]:
kf=KFold(n_splits=10)

In [19]:
fold=1
for train,test in kf.split(x_train):
    print('fold number :',fold)
    
    for i,clf in enumerate(algos):
        print('\n'+'Algo number :',i+1)
        
        x_train_train=x_train.loc[train]
        y_train_train=y_train[train]
        x_train_test=x_train.loc[test]
        
        clf.fit(x_train_train,y_train_train)
        p=clf.predict_proba(x_train_test)[:,1]
        
        layer1.iloc[test,i]=p
    fold+=1

fold number : 1
Algo number : 1
Algo number : 2
Algo number : 3
Algo number : 4
Algo number : 5
fold number : 2
Algo number : 1
Algo number : 2
Algo number : 3
Algo number : 4
Algo number : 5
fold number : 3
Algo number : 1
Algo number : 2
Algo number : 3
Algo number : 4
Algo number : 5
fold number : 4
Algo number : 1
Algo number : 2
Algo number : 3
Algo number : 4
Algo number : 5
fold number : 5
Algo number : 1
Algo number : 2
Algo number : 3
Algo number : 4
Algo number : 5
fold number : 6
Algo number : 1
Algo number : 2
Algo number : 3
Algo number : 4
Algo number : 5
fold number : 7
Algo number : 1
Algo number : 2
Algo number : 3
Algo number : 4
Algo number : 5
fold number : 8
Algo number : 1
Algo number : 2
Algo number : 3
Algo number : 4
Algo number : 5
fold number : 9
Algo number : 1
Algo number : 2
Algo number : 3
Algo number : 4
Algo number : 5
fold number : 10
Algo number : 1
Algo number : 2
Algo number : 3
Algo number : 4
Algo number : 5


In [22]:
layer1

,clf1,clf2,clf3,clf4,clf5
0,0.16,0.135,0.14,0.386297,0.193522
1,0.12,0.010,0.00,0.134280,0.011568
2,0.10,0.000,0.00,0.128753,0.002680
3,0.18,0.165,0.25,0.418074,0.360000
4,0.20,0.015,0.00,0.166988,0.041188
5,0.30,0.000,0.00,0.128753,0.001816
6,0.16,0.155,0.16,0.337832,0.357224
7,0.32,0.850,0.77,0.344730,0.447509
8,0.16,0.015,0.05,0.134280,0.070779
9,0.28,0.000,0.00,0.134280,0.010866


In [20]:
rows=x_test.shape[0]
layer2_test=pd.DataFrame({'clf1':np.zeros(rows),'clf2':np.zeros(rows),'clf3':np.zeros(rows),
                         'clf4':np.zeros(rows),'clf5':np.zeros(rows)})

In [21]:
for i,clf in enumerate(algos):
    print('\n'+'algos number ',i+1)
    clf.fit(x_train,y_train)
    p=clf.predict_proba(x_test)[:,1]
    
    layer2_test.iloc[:,i]=p


algos number  1

algos number  2

algos number  3

algos number  4

algos number  5


In [23]:
layer2_test

,clf1,clf2,clf3,clf4,clf5
0,0.22,0.060,0.08,0.132948,0.051074
1,0.26,0.000,0.02,0.132948,0.031240
2,0.12,0.245,0.28,0.592467,0.581021
3,0.16,0.205,0.29,0.337360,0.305764
4,0.24,0.055,0.02,0.133126,0.028447
5,0.20,0.725,0.66,0.296129,0.543560
6,0.22,0.730,0.77,0.601983,0.758608
7,0.26,0.160,0.14,0.343564,0.122867
8,0.26,0.905,0.91,0.727740,0.975059
9,0.16,0.005,0.02,0.168431,0.043467


In [24]:
logr=LogisticRegression(class_weight='balanced')

In [25]:
logr.fit(layer1,y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [26]:
roc_auc_score(y_test,logr.predict_proba(layer2_test)[:,1])

0.9201212923511819

In [27]:
p=logr.predict_proba(layer2_test)

In [28]:
p

array([[0.87443524, 0.12556476],
       [0.8926103 , 0.1073897 ],
       [0.16779764, 0.83220236],
       ...,
       [0.00924905, 0.99075095],
       [0.90898276, 0.09101724],
       [0.89717498, 0.10282502]])

In [ ]:
#piplines
#pickel files in python
#dask